In [1]:
from src.FunctionsProperties import *
import matplotlib.pyplot as plt

## Create all_files

In [ ]:
# Create folders to Results""
make_results_folders()
#move_to_data()

In [ ]:
def trade_headers(file):
    # Verificar se o arquivo está vazio
    if os.stat(file).st_size == 0:
        os.remove(file)
        print("O arquivo estava vazio e foi excluído.")
        return

    # Carregar o CSV em um DataFrame
    data = pd.read_csv(file, sep=',')

    # Cabeçalhos desejados
    cabecalhos_desejados = {
        '#mean_shortest_path': '#mean shortest path',
        '#diamater': '# diamater',
        '#ass_coeff': '#assortativity coefficient'
    }

    # Verificar se os cabeçalhos precisam ser atualizados
    atualizar = any(col in cabecalhos_desejados for col in data.columns)

    if atualizar:
        data.rename(columns=cabecalhos_desejados, inplace=True)
        # Salvar o DataFrame atualizado
        data.to_csv(file, sep=',', index=False)
        print("Cabeçalhos atualizados com sucesso!")
    else:
        print("Os cabeçalhos já estão corretos. Nenhuma alteração foi feita.")


In [ ]:
import os

def update_header_in_filenames(base_path):
    for root, dirs, files in os.walk(base_path):
        if "filenames.txt" in files:
            file_path = os.path.join(root, "filenames.txt")
            with open(file_path, "r") as f:
                lines = f.readlines()
            
            # Verifica o header atual
            if lines and lines[0].strip() == "filenames":
                print(f"Trocando header em: {file_path}")
                lines[0] = "filename\n"  # Substitui o header
                with open(file_path, "w") as f:
                    f.writelines(lines)
            elif lines and lines[0].strip() == "filename":
                print(f"Header já correto em: {file_path}")
            else:
                print(f"Adicionando header em: {file_path}")
                with open(file_path, "w") as f:
                    f.write("filename\n")
                    f.writelines(lines)

# Caminho base da estrutura de pastas
base_path = "../../data"
update_header_in_filenames(base_path)


In [ ]:
def clean_properties_file(N, dim, alpha_a, alpha_g):
    path_d = f"../../data/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
    # Ler o arquivo e armazenar as linhas válidas
    with open(path_d, "r") as file:
        lines = file.readlines()

    # Verificar se as linhas indesejadas estão presentes
    filtered_lines = [line for line in lines if line.strip() != "diamater #assortativity coefficient"]

    # Se houve modificação, reescrever o arquivo
    if len(filtered_lines) != len(lines):
        with open(path_d, "w") as file:
            file.writelines(filtered_lines)

    print("Arquivo processado com sucesso!")


In [2]:
# ==> READ DOCUMENTATION OF FUNCTION all_properties_file(N,dim, alpha_a, alpha_g) BEFORE RUN THAT CELL!! <==
folder_data = "../../data"
all_combinations_parms =  extract_alpha_values(folder_data)
#update_headers(folder_data)
N_ter = len(all_combinations_parms)# for file in all_files:


i = 0
for parm in all_combinations_parms:
    # if(parm[0]==160000):
    #     N, dim, alpha_a, alpha_g = parm[0], parm[1], parm[2], parm[3]
    #     file_path = f"../../data_2/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/prop"
    N, dim, alpha_a, alpha_g = parm[0], parm[1], parm[2], parm[3]
    
    #clean_properties_file(N, dim, alpha_a, alpha_g)
    # file_path = f"../../data/N_{N}/dim_{dim}/alpha_a_{alpha_a:.2f}_alpha_g_{alpha_g:.2f}/prop"
    # all_files = glob.glob(os.path.join(file_path,"*.csv"))
    # i += 1
    # print(f"faltam {N_ter - i} termos, N = {N}, dim = {dim}, alpha_a = {alpha_a:.2f}, alpha_g = {alpha_g:.2f}")
    # for file in all_files:
    #     if os.stat(file).st_size == 0:
    #         os.remove(file)
    #         pass
    #     else:
    #         trade_headers(file)

    print(f"{parm[0]}, {parm[1]}, {parm[2]:.2f}, {parm[3]}")
    all_properties_file(parm[0], parm[1], parm[2], parm[3])
    if( i / 10 - int(i / 10) == 0.0):
        clear_output()    
    remove_outliers(N, dim, alpha_a, alpha_g)


In [3]:
N = [5000, 10000, 20000, 40000, 80000, 100000 ,160000, 320000]
folder_data = "../../data"
dim = [1, 2, 3, 4]
#format_file(N, dim)
all_data(folder_data)

N = 160000, dim = 1, alpha_a = 1.50, alpha_g = 2.00
N = 160000, dim = 1, alpha_a = 11.00, alpha_g = 2.00
N = 160000, dim = 1, alpha_a = 2.00, alpha_g = 4.00
N = 160000, dim = 1, alpha_a = 9.25, alpha_g = 2.00
N = 160000, dim = 1, alpha_a = 2.75, alpha_g = 2.00
N = 160000, dim = 1, alpha_a = 2.00, alpha_g = 1.00
N = 160000, dim = 1, alpha_a = 2.50, alpha_g = 2.00
N = 160000, dim = 1, alpha_a = 10.25, alpha_g = 2.00
❌ Removendo pasta vazia e sem dados: ../../data/N_160000/dim_1/alpha_a_0.65_alpha_g_2.00/gml
❌ Removendo pasta vazia e sem dados: ../../data/N_160000/dim_1/alpha_a_0.65_alpha_g_2.00/prop
❌ Removendo pasta vazia e sem dados: ../../data/N_160000/dim_1/alpha_a_0.65_alpha_g_2.00
N = 160000, dim = 1, alpha_a = 2.00, alpha_g = 16.00
N = 160000, dim = 1, alpha_a = 8.00, alpha_g = 2.00
N = 160000, dim = 1, alpha_a = 2.00, alpha_g = 2.00
N = 160000, dim = 1, alpha_a = 4.00, alpha_g = 2.00
N = 160000, dim = 1, alpha_a = 2.00, alpha_g = 5.00
N = 160000, dim = 1, alpha_a = 5.25, alpha_g 

# <b> Statistical Analysis </b>

### For some reason, for some sets of parameters ($N$, $dim$, $\alpha_a$, $\alpha_g$) , the values ​​of the assortativity coefficients have a large fluctuation (large STD values), generating averages with a considerably large standard error. With this in mind, it is necessary to perform an analysis and filtering on the data whose sets present these problems.

## <b> Using an absolute approach </b>

### Set absolute bounds for the standard error and standard deviation ($\sigma$), regardless of the measurement, and filter out data that exceed these bounds. In my case, i'll use std for other combinations ($\alpha_g$, $\alpha_a$) where the standard error are smaller 10%, when $N$ and $dim$ are equals. The process is very simple. Given the set $R$ $=$ $[-0.0245503$, $-0.0155637$, $-0.0219698$, $-0.0144903$ $,...]$ and a good value for $\sigma$, $\sigma$ $=$ $\sigma_{ideal}$. Each value outside of range $-\sigma_{ideal}$ $<$ $value$ $<$ $\sigma_{ideal}$ or $value < |\sigma_{ideal}|$ is refused.

In [ ]:
# Import dataframe with all data
df = pd.read_csv("../../data/all_data.txt", delimiter=" ")
# Filter all data for alpha_a > 1.0
df = df[df["alpha_g"]>=1.0]

# Removing duplicate lines
df = df.drop_duplicates()
df = df.reset_index(drop=True)

# Save new dataframe
df.to_csv("../../data/all_data.txt", index=False, sep=' ')
df.head()

In [ ]:
np.sort(df[df["N"]==160000]["alpha_a"].unique())

In [ ]:
alpha_a_all = [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.25, 5.75]
alpha_g = 2.0
dim = [1,2,3,4]
n = 160000
n_s = 10000

parms_need = []
for aa in alpha_a_all:
    for d in dim:
        parms_need.append((n, d, aa, alpha_g))

df_5k = df[df["N"]==n]
df_5k = df_5k[df_5k["alpha_g"]==2.0]
df_5k

parms_have = []
for index, row in df_5k.iterrows():
    parms_have.append((n, int(row['dim']), float(row['alpha_a']), alpha_g))


# Converter listas para conjuntos e calcular a diferença
parms_missing = set(parms_need) - set(parms_have)

# Exibir os elementos que estão em parms_need mas não em parms_have
print("Elementos que faltam:", list(parms_missing))


In [ ]:
# Filter N to future linear regression Propetie = Xi_Propertie(dim, alpha_a, alpha_g) * log_10(N) + Chi_Propertie(dim, alpha_a, alpha_g)
N = [5000, 10000, 20000, 40000, 80000, 160000, 320000]
alpha_filter = [0.0, 1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]
dimensions = [1, 2, 3, 4]

# Filtered dataframe with these sets
f_df = df[
    (df['N'].isin(N)) &
    (df['alpha_a'].isin(alpha_filter)) &
    (df['dim'].isin(dimensions)) &
    (df['alpha_g'] == 2.0)
].sort_values(by=["alpha_a"]).reset_index(drop=True)
f_df

In [ ]:
# Starting with empty dictionary
data_percent = {"N": [], "dim": [], "alpha_a": [], "alpha_g": [], "N_samples":[] ,"per_short": 
                [],"std_short":[] ,"per_ass": [],"std_ass":[] ,"per_dia": [], "std_dia":[]}

# Iterate through the rows of the DataFrame
for _, row in f_df.iterrows():
    data_percent["N"].append(row["N"])
    data_percent["dim"].append(row["dim"])
    data_percent["alpha_a"].append(row["alpha_a"])
    data_percent["alpha_g"].append(row["alpha_g"])
    data_percent["N_samples"].append(row["N_samples"])
    
    data_percent["std_short"].append(row["short_std"])
    data_percent["std_ass"].append(row["ass_coeff_std"])
    data_percent["std_dia"].append(row["diameter_std"])
    
    # calculating percentages of each standard error
    per_short = (row["short_err"] / row["short_mean"]) * 100 if row["short_mean"] != 0 else 0
    per_ass = (row["ass_coeff_err"] / abs(row["ass_coeff_mean"])) * 100 if row["ass_coeff_mean"] != 0 else 0
    per_dia = (row["diameter_err"] / row["diameter_mean"]) * 100 if row["diameter_mean"] != 0 else 0
    
    data_percent["per_short"].append(per_short)
    data_percent["per_ass"].append(per_ass)
    data_percent["per_dia"].append(per_dia)

# New dataframe with all percentages to each properties
df_per = pd.DataFrame(data=data_percent)
df_per.to_csv("percent_err.txt", index=False, sep=' ')
df_per.head()

In [ ]:
# Filtering just percentages with values greater than 10
filtered_df = df_per[
    (df_per['per_short'] >= 10) | 
    (df_per['per_ass'] >= 10) | 
    (df_per['per_dia'] >= 10)
].reset_index(drop=True)

In [ ]:
filtered_df.head()

In [ ]:
# Just assortativity with % standard error >= 10%
filtered_df[filtered_df["per_short"]>=10],filtered_df[filtered_df["per_dia"]>=10], filtered_df[filtered_df["per_ass"]>=10]

In [ ]:
# Filtering the general dataframe (df), with parameters other than those whose error is greater than 10%
N_filt = [i for i in filtered_df["N"]]
dim_filt = [i for i in filtered_df["dim"]]
alpha_a_filt = [i for i in filtered_df["alpha_a"]]
alpha_g_filt = [i for i in filtered_df["alpha_g"]]
N_samples_filt = [i for i in filtered_df["N_samples"]]

comb = {"N":N_filt, "dim": dim_filt, "alpha_a":alpha_a_filt, "alpha_g":alpha_g_filt, "N_samples":N_samples_filt}
df_remover = pd.DataFrame(comb)

df_non_10 = df.merge(df_remover, on=['N', 'dim', 'alpha_a', 'alpha_g', "N_samples"], how='left', indicator=True)
df_non_10 = df_non_10[df_non_10['_merge'] == 'left_only'].drop(columns=['_merge'])

In [ ]:
df_non_10[df_non_10["N"]==40000]

In [ ]:
# Dataframe with % error in assortativity greater 10%
filtered_df[filtered_df["N"]==5000]

In [ ]:
data_parameters = {"N": [], "dim": [], "alpha_a": [], "alpha_g": [], "N_samples":[]}

for _, row in filtered_df.iterrows():
    data_parameters["N"].append(int(row["N"]))
    data_parameters["dim"].append(int(row["dim"]))
    data_parameters["alpha_a"].append(float(row["alpha_a"]))
    data_parameters["alpha_g"].append(float(row["alpha_g"]))
    if(row["N"] == 5000):
        data_parameters["N_samples"].append(20000)
    elif(row["N"] == 10000):
        data_parameters["N_samples"].append(3200)
    elif(row["N"] == 20000):
        data_parameters["N_samples"].append(700)
    elif(row["N"] == 40000):
        data_parameters["N_samples"].append(500)
    elif(row["N"] == 80000):
        data_parameters["N_samples"].append(200)
    elif(row["N"] == 160000):
        data_parameters["N_samples"].append(30)
    elif(row["N"] == 320000):
        data_parameters["N_samples"].append(15)

df_run_multi = pd.DataFrame(data=data_parameters)
df_run_multi.to_csv("../python/run_multi.txt", index=False, sep=' ')

In [ ]:
parameters = []
combinations = extract_alpha_values2("../../data_2")
for parms in combinations:
    N, dim, alpha_a, alpha_g = parms[0], parms[1], parms[2], parms[3]
    files = f"../../data_2/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
    if(os.path.exists(files)):
        df = pd.read_csv(files,delimiter=' ')
        if "#ass_coeff" in df.columns:
            R = df["#ass_coeff"]
            r_per = abs(R.sem() / R.mean()) * 100
            #all_properties_file2(N, dim, alpha_a, alpha_g)
            
            #if(N==320000):
                #print(f"dim = {dim}, alpha_a = {alpha_a}, alpha_g = {alpha_g}, N = {N}, N_samples = {len(R)}, r_per = {r_per}")
            if (30 > r_per > 10):
                if (N==320000):
                    #all_properties_file2(N, dim, alpha_a, alpha_g)
                    #remove_outliers(N, dim, alpha_a, alpha_g)
                    print(f"dim = {dim}, alpha_a = {alpha_a}, alpha_g = {alpha_g}, N = {N}, N_samples = {len(R)}, r_per = {r_per}")
    # else:
    #     all_files = glob.glob(os.path.join(f"../../data_2/N_{N}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/", "*.csv"))
    #     #for file in all_files:
    #         #trade_headers(file)
    #     all_properties_file2(N, dim, alpha_a, alpha_g)

In [ ]:
# Dataframe with % error in assortativity less than 10%
df_non_10

In [ ]:
N = [5000, 10000, 20000, 40000, 80000, 160000, 320000]
# get σ_ideal
mean_std_N_for_dim = {"N":[], "dim":[], "ass_coeff_std_mean":[]}
for n in N:
    for dim in dimensions:
        df_N_dim = df_non_10[(df_non_10["N"]==n) & (df_non_10["dim"]==dim) & (df_non_10["alpha_g"]==2.0)]
        
        mean_std_N_for_dim["N"].append(n)
        mean_std_N_for_dim["dim"].append(dim)
        mean_std_N_for_dim["ass_coeff_std_mean"].append(df_N_dim["ass_coeff_std"].mean())

# list of mean σ to set with σ_ideal smallest 10% for each combinations (N, dim)
df_std_N_dim = pd.DataFrame(data=mean_std_N_for_dim)

In [ ]:
# set with σ_ideal for each (N, dim)
df_std_N_dim

In [ ]:
df_5 = filtered_df[filtered_df["N"]==5000]
df_10 = filtered_df[filtered_df["N"]==10000]
df_20 = filtered_df[filtered_df["N"]==20000]
df_40 = filtered_df[filtered_df["N"]==40000]
df_80 = filtered_df[filtered_df["N"]==80000]
df_160 = filtered_df[filtered_df["N"]==160000]
df_320 = filtered_df[filtered_df["N"]==320000]
#N = 10000,  n_s 3200
#N = 20000,  n_s 700
#N = 40000,  n_s 500
#N = 80000,  n_s 200
#N = 160000, n_s 30
#N = 320000, n_s 15

In [ ]:
N = [5000, 10000, 20000, 40000, 160000]
count = 0
for n in N:
    df_f = filtered_df[filtered_df["N"]==n]
    for _, row in df_f.iterrows():
        dim = int(row["dim"])
        alpha_a, alpha_g = row["alpha_a"], row["alpha_g"]
        path_file = f"../../data_2/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
        source_path = f"../../data_2/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/"
        df = pd.read_csv(path_file, sep=' ')
        R = df["#ass_coeff"]
        R_per = (R.sem()/abs(R.mean()))*100
        if(R_per > 10):
            count +=1
            print(f"alpha_a = {alpha_a}, alpha_g = {alpha_g}, dim = {dim}, err = {R_per}, N = {n}, N_s = {len(R)}")
print(count)

In [ ]:
filtered_df

In [ ]:
filtered_df

In [ ]:
def extract_alpha_values2(folder_data):
    # Caminho inicial
#    base_path = "../../data_2"
    base_path = folder_data

    # Regex para capturar nvalue, dvalue, alpha_a (aavalue) e alpha_g (agvalue)
    pattern = r"N_(\d+)/dim_(\d+)/alpha_a_([\d.]+)_alpha_g_([\d.]+)"

    # Estrutura para armazenar as combinações encontradas
    combinations = set()

    # Percorrer todas as subpastas a partir de base_path
    for root, dirs, files in os.walk(base_path):
        match = re.search(pattern, root)
        if match:
            nvalue = int(match.group(1))  # nvalue como inteiro
            dvalue = int(match.group(2))  # dvalue como inteiro
            aavalue = float(match.group(3))  # alpha_a como float
            agvalue = float(match.group(4))  # alpha_g como float
            combinations.add((nvalue, dvalue, aavalue, agvalue))
    return combinations

In [ ]:
all_parms = extract_alpha_values2("../../data_2")
for parms in all_parms:
    path_prop = f"../../data_3/N_{parms[0]}/dim_{parms[1]}/alpha_a_{parms[2]}_alpha_g_{parms[3]}/prop"
    # Verificar se a pasta existe
    if os.path.exists(path_prop):
        # Listar todos os arquivos na pasta
        for file_name in os.listdir(path_prop):
            file_path = os.path.join(path_prop, file_name)
            # Verificar se é um arquivo antes de excluir
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Arquivo '{file_path}' removido com sucesso.")
        print(f"Todos os arquivos em '{path_prop}' foram removidos.")
    else:
        print(f"A pasta '{path_prop}' não existe.")

In [ ]:
n = 40000
dim, alpha_a, alpha_g = 4, 10.0, 2.0
all_properties_file2(n, dim, alpha_a, alpha_g)

In [ ]:


path_file = f"../../data_2/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
# Criar um DataFrame
df = pd.read_csv(path_file, sep=' ')
R = df["#ass_coeff"]
r_mean, r_std, r_err = R.mean(), R.std(), R.sem()
fig, ax = plt.subplots(figsize=(14, 10))

ax.errorbar(r_mean,1,xerr=r_std,marker='o',lw=1.4, capsize=10, capthick=1.4)
R_in = [i for i in R if abs(i-r_mean) <= r_std]
R_out = [i for i in R if abs(i-r_mean) > r_std]

ax.plot(R_in,np.ones(len(R_in)),'o', color='blue')
ax.plot(R_out,np.ones(len(R_out)),'o', color='green')
#ax.plot(R,np.ones(len(R)),'o')
#ax.set_xlim([r_mean - r_err, r_mean + r_err])

std_in = np.std(np.array(R_in), ddof=1)

R_in_err = std_in / np.sqrt(len(R_in))

print("R_size:",len(R))
print("R_in_size:",len(R_in))
print("R_out_size:",len(R_out))
print("R_ratio:",len(R_in)/len(R_out))
print("R_err:", (r_err/abs(r_mean))*100)
print("R_mean_before:", r_mean)
print("R_mean_after:", np.mean(np.array(R_in)))
print("R_err_after:", (R_in_err) / abs(np.mean(R_in))*100 )
plt.show()

In [ ]:
n, dim, alpha_a, alpha_g = 5000, 1, 3.0, 2.0
file_path = f"../../data_2/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
# Criar um DataFrame
#df = pd.read_csv(path_file, sep=' ')
df = pd.read_csv(file_path, delim_whitespace=True)
R = df["#ass_coeff"]
print(df)
num_samples = float(len(R))
num_points = 20
divisions = int(num_samples // num_points)
width = [(0,i*divisions) for i in range(1,num_points)]

R_err = []
R_mean = []
R_std = []
n_samples = []
C_V = []

for i in range(len(width)):
    R_err.append(R[0:width[i][1]].sem())
    R_mean.append(R[0:width[i][1]].mean())
    
    r_std = abs(R[0:width[i][1]].std())
    r_mean = abs(R[0:width[i][1]].mean())
    R_std.append(r_std)
    #print(r_std, r_mean)
    #C_V.append((/)*100)
    n_samples.append(len(R[0:width[i][1]]))

#print(R_std)
#print(R_mean)
#print(R_err)
print("N_samples:", len(R))
print(f"R_err = {abs(R.sem()/R.mean())*100} %" )
percent = [abs(i/j)*100 for i,j in zip(R_err, R_mean)]
#print([i/j for i,j in zip(R_err, R_mean)])
#plt.plot(n_samples, R_err,'o', label='errors')
plt.plot(n_samples, percent,'o', label=f'errors, err_minimum = {min(percent)} %')
plt.axhline(y=10, color='red', linestyle='--', linewidth=2, label="y=10%")  # Linha horizontal
#plt.plot(n_samples, R_mean,'o', label='means')
#plt.plot(n_samples, C_V,'o', label='means')
#plt.axhline(y=np.mean(np.array(C_V)), xmin=min(n_samples),xmax=max(n_samples))
plt.ylabel("R_error(%)")
plt.xlabel("N_samples")
plt.ylim([0, 100])
plt.xlim([0, width[-1][1]])
plt.legend()
#plt.xscale("log")
#plt.savefig("CV_n_samples.pdf")
plt.show()

In [ ]:
N, dim, alpha_a, alpha_g = 5000, 3, 12.0, 2.0
all_properties_file2(N, dim, alpha_a, alpha_g)

In [ ]:
n, dim, alpha_a, alpha_g = 40000, 4, 10.0, 2.0
file_path = f"../../data_2/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
# Criar um DataFrame
#df = pd.read_csv(path_file, sep=' ')
df = pd.read_csv(file_path, delim_whitespace=True)
R = df["#ass_coeff"]

num_samples = float(len(R))
num_points = 20
divisions = int(num_samples // num_points)
width = [(0,i*divisions) for i in range(1,num_points)]

R_err = []
R_mean = []
R_std = []
n_samples = []
C_V = []

for i in range(len(width)):
    R_err.append(R[0:width[i][1]].sem())
    R_mean.append(R[0:width[i][1]].mean())
    
    r_std = abs(R[0:width[i][1]].std())
    r_mean = abs(R[0:width[i][1]].mean())
    R_std.append(r_std)
    #print(r_std, r_mean)
    #C_V.append((/)*100)
    n_samples.append(len(R[0:width[i][1]]))

#print(R_std)
#print(R_mean)
#print(R_err)
print("N_samples:", len(R))
print(f"R_err = {abs(R.sem()/R.mean())*100} %" )
percent = [abs(i/j)*100 for i,j in zip(R_err, R_mean)]
#print([i/j for i,j in zip(R_err, R_mean)])
#plt.plot(n_samples, R_err,'o', label='errors')
plt.plot(n_samples, percent,'o', label=f'errors, err_minimum = {min(percent)} %')
plt.axhline(y=10, color='red', linestyle='--', linewidth=2, label="y=10%")  # Linha horizontal
#plt.plot(n_samples, R_mean,'o', label='means')
#plt.plot(n_samples, C_V,'o', label='means')
#plt.axhline(y=np.mean(np.array(C_V)), xmin=min(n_samples),xmax=max(n_samples))
plt.ylabel("R_error(%)")
plt.xlabel("N_samples")
plt.ylim([0, 100])
plt.xlim([0, width[-1][1]])
plt.legend()
#plt.xscale("log")
#plt.savefig("CV_n_samples.pdf")
plt.show()

In [ ]:
filtered_df[filtered_df["N"]==10000].reset_index(drop=True)



In [ ]:
filtered_df

In [ ]:
n, dim, alpha_a, alpha_g = 5000, 3, 12.0, 2.0
file_path = f"../../data_2/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
# Criar um DataFrame
#df = pd.read_csv(path_file, sep=' ')
df = pd.read_csv(file_path, sep=' ')
R = df["#ass_coeff"]
print(df)
num_samples = float(len(R))
num_points = 20
divisions = int(num_samples // num_points)
width = [(0,i*divisions) for i in range(1,num_points)]

R_err = []
R_mean = []
R_std = []
n_samples = []
C_V = []

for i in range(len(width)):
    R_err.append(R[0:width[i][1]].sem())
    R_mean.append(R[0:width[i][1]].mean())
    
    r_std = abs(R[0:width[i][1]].std())
    r_mean = abs(R[0:width[i][1]].mean())
    R_std.append(r_std)
    #print(r_std, r_mean)
    #C_V.append((/)*100)
    n_samples.append(len(R[0:width[i][1]]))

#print(R_std)
#print(R_mean)
#print(R_err)
print("N_samples:", len(R))
print(f"R_err = {abs(R.sem()/R.mean())*100} %" )
percent = [abs(i/j)*100 for i,j in zip(R_err, R_mean)]
#print([i/j for i,j in zip(R_err, R_mean)])
#plt.plot(n_samples, R_err,'o', label='errors')
plt.plot(n_samples, percent,'o', label=f'errors, err_minimum = {min(percent)} %')
plt.axhline(y=10, color='red', linestyle='--', linewidth=2, label="y=10%")  # Linha horizontal
#plt.plot(n_samples, R_mean,'o', label='means')
#plt.plot(n_samples, C_V,'o', label='means')
#plt.axhline(y=np.mean(np.array(C_V)), xmin=min(n_samples),xmax=max(n_samples))
plt.ylabel("R_error(%)")
plt.xlabel("N_samples")
#plt.ylim([0, 100])
plt.xlim([0, width[-1][1]])
plt.legend()
#plt.xscale("log")
#plt.savefig("CV_n_samples.pdf"')
plt.show()

In [ ]:
n = 5000
dim, alpha_a, alpha_g = 1, 8.0, 3.0
path_file = f"../../data/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
# Criar um DataFrame
df = pd.read_csv(path_file, sep=',')
R = df["#ass_coeff"]

r_mean, r_std, r_err = R.mean(), R.std(), R.sem()
fig, ax = plt.subplots(figsize=(14, 10))

ax.errorbar(r_mean,1,xerr=r_std,marker='o',lw=1.4, capsize=14 ,capthick=1.4)

R_in = [i for i in R if abs(i-r_mean) <= r_std]
R_out = [i for i in R if abs(i-r_mean) > r_std]

ax.plot(R_in,np.ones(len(R_in)),'o', color='blue')
ax.plot(R_out,np.ones(len(R_out)),'o', color='green')

std_in = np.std(np.array(R_in), ddof=1)

R_in_err = std_in / np.sqrt(len(R_in))

print("R_size:",len(R))
print("σ:", r_std)
print("R_mean:", r_mean)
print("R_in_size:",len(R_in))
print("R_out_size:",len(R_out))
print("R_ratio:",len(R_in)/len(R_out))
print("R_err:", (r_err/abs(r_mean))*100)
print("R_mean_in:", np.mean(np.array(R_in)))

print("R_err_int:", (R_in_err) / abs(np.mean(R_in))*100 )
#ax.set_xlim([r_mean - r_err, r_mean + r_err])
plt.show()

In [ ]:
filtered_df

In [ ]:
# box_plot
n=320000
dim = 1
alpha_a, alpha_g = 0.0, 2.0
path_file = f"../../data/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
a = pd.read_csv(path_file, sep=',')

plt.figure(figsize=(8, 6))
plt.boxplot(a["#ass_coeff"], vert=True, patch_artist=True, showmeans=True)
plt.title("Box Plot for #ass_coeff", fontsize=14)
plt.ylabel("#ass_coeff", fontsize=12)
plt.show()

In [ ]:
filtered_df[filtered_df["N"]==320000]

In [ ]:
N = 20000
dim = [2, 3, 4, 3, 4, 4]
alpha_a = [6.0, 9.0, 10.0, 10.0, 11.0, 12.0]
alpha_g = 2.0

In [ ]:
n, dim, alpha_a, alpha_g = 40000, 3, 8.0, 2.0
file_path = f"../../data_2/N_{n}/dim_{dim}/alpha_a_{alpha_a}_alpha_g_{alpha_g}/properties_set.txt"
# Criar um DataFrame
#df = pd.read_csv(path_file, sep=' ')
df = pd.read_csv(file_path, sep=' ')
R = df["#ass_coeff"]
print(df)
num_samples = float(len(R))
num_points = 20
divisions = int(num_samples // num_points)
width = [(0,i*divisions) for i in range(1,num_points)]

R_err = []
R_mean = []
R_std = []
n_samples = []
C_V = []

for i in range(len(width)):
    R_err.append(R[0:width[i][1]].sem())
    R_mean.append(R[0:width[i][1]].mean())
    
    r_std = abs(R[0:width[i][1]].std())
    r_mean = abs(R[0:width[i][1]].mean())
    R_std.append(r_std)
    #print(r_std, r_mean)
    #C_V.append((/)*100)
    n_samples.append(len(R[0:width[i][1]]))

#print(R_std)
#print(R_mean)
#print(R_err)
print("N_samples:", len(R))
print(f"R_err = {abs(R.sem()/R.mean())*100} %" )
percent = [abs(i/j)*100 for i,j in zip(R_err, R_mean)]
#print([i/j for i,j in zip(R_err, R_mean)])
#plt.plot(n_samples, R_err,'o', label='errors')
plt.plot(n_samples, percent,'o', label=f'errors, err_minimum = {min(percent)} %')
plt.axhline(y=10, color='red', linestyle='--', linewidth=2, label="y=10%")  # Linha horizontal
#plt.plot(n_samples, R_mean,'o', label='means')
#plt.plot(n_samples, C_V,'o', label='means')
#plt.axhline(y=np.mean(np.array(C_V)), xmin=min(n_samples),xmax=max(n_samples))
plt.ylabel("R_error(%)")
plt.xlabel("N_samples")
#plt.ylim([0, 100])
plt.xlim([0, width[-1][1]])
plt.legend()
#plt.xscale("log")
#plt.savefig("CV_n_samples.pdf"')
plt.show()